# #FP [How to design co-programs](https://youtu.be/w2XCnbLBHmw?si=iBX8L96XXK_B7F3O)

Inspire by the book: [How to Design Programs](https://htdp.org/2023-8-14/Book/index.html)

- motivating problem

## Where do programs come from?

- start by analysing the structure of the input
  - input can be a list

- HtDP
  - input  
  - composite
    - mixed
    - inductive
  - skeleton
    - name the component
    - enumerate the alternatives
    - use structural recursion

| input | program skeleton |
|-------|------------------|
| compositie(record) |name the component |
| mixed(union) | enumerate the alternatives|
| inductive | use structural recursion|

## Example (poker deck)

In [9]:
(* name the component *)
type suit = unit
type rank = unit
type cardR = {
  rank : rank;
  suite : suit
}

type card = | Regular of cardR | Joker


type suit = unit


type rank = unit


type cardR = { rank : rank; suite : suit; }


type card = Regular of cardR | Joker


## Example for Inductive Data


In [22]:
(* insertion sort *)
let insert a = function
  | [] -> [a]
  | x::xs when a < x -> a :: x :: xs
  | x::xs -> x::(insert a xs)

let rec isort = function
  | [] -> []
  | x::xs -> insert x (isort xs)


let sorted_list = isort [4;3;2;5;1;6;2]

val insert : 'a -> 'a list -> 'a list = <fun>


val isort : 'a list -> 'a list = <fun>


val sorted_list : int list = [1; 2; 2; 3; 4; 5; 6]


| output | program skeleton |
|-------|------------------|
| compositie(record) |name the component |
| mixed(union) | enumerate the alternatives|
| inductive | use structural corecursion|

## What's structural corecursion

```Haskell
h z | ... = []
    | otherwise = x:xs
    where x = ... z ..
          xs = h (...z...)
```

this is an unfold

## Example of structural corecursion


In [26]:
(* selection sort *)
let min = function
  | [] -> failwith "invalid input"
  | x::xs -> List.fold_left (fun acc y -> if acc < y then acc else y) x xs

let rec taken_elem a = function
  | [] -> []
  | x::xs when a = x -> xs
  | x::xs -> x :: (taken_elem a xs)

let rec ssort = function
  | [] -> []
  | input -> let x = min input in
             let xs  = (taken_elem x input) in
             x::xs

let sorted_list = isort [4;3;2;5;1;6;2]

val min : 'a list -> 'a = <fun>


val taken_elem : 'a -> 'a list -> 'a list = <fun>


val ssort : 'a list -> 'a list = <fun>


val sorted_list : int list = [1; 2; 2; 3; 4; 5; 6]


More tools means more options. 

## Motivating example

zipping two list

- there're cases where the design recipe on input analysis doesn't work
- while the design recipe on output anaysis works

In [31]:
(* based on the input analysis *)
let rec zip xs ys = 
  match xs with
  | [] -> []
  | x::xs ->
    begin
      match ys with
      | [] -> []
      | y::ys -> (x, y) :: zip xs ys
    end

let is_empty = function [] -> true | _ -> false
(* based on the output analysis *)
let rec zip xs ys = 
  if is_empty xs || is_empty ys then
    []
  else
    (List.hd xs, List.hd ys) :: zip (List.tl xs) (List.tl ys)


val zip : 'a list -> 'b list -> ('a * 'b) list = <fun>


val is_empty : 'a list -> bool = <fun>


val zip : 'a list -> 'b list -> ('a * 'b) list = <fun>


In [ ]:
## Generative recursion

- structural recursive
  - subproblem is projection 
  - triavial computation like head/tail
- generative recursion
  - subproblem are computed instead of projected

In [36]:
(* example generative recursion *)
let rec qsort =
  function
  | [] -> []
  | x::xs ->
    let less_than = List.filter ((>=) x) xs in
    let more_than = List.filter ((<) x) xs in
    (qsort less_than) @ [x] @ (qsort more_than)

let sorted_list = qsort [4;3;2;5;1;6;2]

val qsort : 'a list -> 'a list = <fun>


val sorted_list : int list = [1; 2; 2; 3; 4; 5; 6]


However, the qsort demonstrated as an example for generative recursion suffers the problem of bad performance. It's short but slow.

## Combing structural recursion and corecursion

generative recursion has a creative step. for example
1. tree structure in the quick sort

#TLDR Jeremey said generative recursion is not ad-hoc activity. It's in fact a structural activity. 
Yes, it do requires some insight on coming up with intermediate data strucutres which is not related to input data and output data of the program but
it's then a structural programming (structural recursion/corecursion) on the intermediate data structures.

## Separation of concerns

Deeforestation
- a program transformation to eliminate intermediate lists or tree structures that are created and then immediately consumed by a program.

More about generative recursion.

separates parts may be studied separately.

1. corecrusion: input => middle, recusion: middle => output
   * fold after unfold - hylomorphism
2. recursion: input => middle, corecursion: middle => outptu
   * unfold after fold - metamorphism

## A learning opportunity

* #TLDR data strucutres determine the program structures

using merge sort as an example. Represent tree the following way produces awkward programs.

In [63]:
type tree =
  | Empty
  | Node of (tree * int * tree)

let halves input =  
  let target = (List.length input) / 2 in
  let start_acc = (([], []), 0) in
  let fold_action elem ((left, right), count) = 
    if count >= target then
      ((elem::left, right), count + 1)
    else
      ((left, elem::right), count + 1)
    in
  List.fold_right fold_action input start_acc |> fst

let rec split = function
  | [] -> Empty
  | x::xs -> let (left, right) = halves xs in
  Node (split left, x, split right)

let rec mergeAll = function
  | Empty -> []
  | Node (left, a, right) -> List.merge compare (mergeAll left) (List.merge compare [a] (mergeAll right))

let msort input = input |> split |> mergeAll

let sorted_list = msort [4;3;2;5;1;6;2]

type tree = Empty | Node of (tree * int * tree)


val halves : 'a list -> 'a list * 'a list = <fun>


val split : int list -> tree = <fun>


val mergeAll : tree -> int list = <fun>


val msort : int list -> int list = <fun>


val sorted_list : int list = [1; 2; 2; 3; 4; 5; 6]


```OCaml
type 'a tree =
    | Leaf of 'a
    | Node of ('a tree, 'a tree)
```

## Laziness

Laziness give automatics deforestation

## Conclusion

1. program strucutre follows data structure
2. recursive programs for recursive data